In [ ]:
%run ../00_AdvancedPythonConcepts/talktools.py

# Parallelism

Python Computing for Data Science (AY250)

## Outline for Today

- Motivation

- Single-machine
    - threading
    - multiprocessing
    - joblib
    - dask
- Multi-machine
    - ipython cluster

## Motivation

Generally, the goal of your computing task is to finish as quickly as possible. The **speed of your processor** at executing instructions and the **speed at which data can be read from disk and from RAM** are major contributors to the execution time. Obviously the **choice of algorithm(s)** is critical too. Choosing an $N \log N$ algorithm over a $N^2$ one that gets the same answer is almost always preferred for any sizeable $N$.

### Types of Bottlenecks

If you think of your run-time program as stream of data and computation on data, it should be clear that **bottlenecks are inevitable**. Your job (as you begin optimize for execution time) is to understand where those bottlenecks are and to use the tools we have in Python to minimize those. (Ultimately, it's a never-ending whack-a-mole).

#### I/O Bound

*"a condition in which the time it takes to complete a computation is determined principally by the period spent waiting for input/output operations to be completed."* -- wikipedia

This can be because we're waiting for a response from an external source (e.g. loading a webpage) or because data needs to be moved around on your bus and we're waiting for it to show up in the right place to compute on. If you have very fast CPUs, you're more likely to be I/O bound.

#### CPU Bound

*"when the time for it to complete a task is determined principally by the speed of the central processor: processor utilization is high, perhaps at 100% usage for many seconds or minutes."* -- wikipedia

If you're doing  algorithmic computations where the amount if input data is small and the amount of output data is also small (e.g. fournier transform) you'll typically be CPU bound. Slowed CPUs lead to more CPU bound bottlenecks. If you have a lot of data ("big") you're moving data around from disk, RAM, cache and you're likely I/O bound.

#### (Memory Bound)

"time to complete a given computational problem is decided primarily by the amount of memory required to hold data" - wikipedia.

<img src="https://www.evernote.com/l/AUUzntxvU9BHWJMZSH_CL3S7YRUjThJTrPEB/image.png">

Source: http://www.slideshare.net/ManojitNandi/parallel-programming-in-python-speeding-up-your-analysis

# Processes & Threads

Each Python interpreter runs in a `process,` containing the program code, stack, and its current activity. 

In [1]:
import os
os.getpid()

7190

Within a process one can create a set of `threads` which share everything with the process in which they were spawned (memory, data, state). But, most generally, they are little programs (with their own stack) that execute `concurrently` (independent of each other). Since they share things like memory, it requires the programmer to "lock" everything that might conflict. The way we make many threads in Python is using the `threading` module.

<div class="alert alert-info">The Global Interpreter Lock (GIL) in Python stops threads from truly happening in parallel. That is, the interpreter can only operate one thread at a time. This is an impliementation detail of how CPython was programmed. Many things you use push threads down into the C-layer and "avoid the GIL". </div>

You can also make many processes, which are copies of the original parent process (memory, data, state) and act independently of each other. To share data between them you have to explicitly do that within each process. The Pythonic way we do multiprocessing (creation of new processes, communication between processes) is with `multiprocessing`.

The goal of computing with `threading` and `multiprocessing` is to not wait around: the CPU should not be idle if it doesn't have too. AND since we almost always have multiple cores, we should be able to let the work we want to do happen in parallel over those cores.

### Threading

`threading.Thread(target=f, args=(...))` is the basic way to use function `f` with arguments in a thread.

`.start()`: Calls the `.run()` of a thread object. This method will raise a `RuntimeError` if called more than once on the same thread object.

In [2]:
import threading

def worker(num):
    """thread worker function"""
    print('Worker: %s' % num)
    return

threads = []
for i in range(5):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

Worker: 0
Worker: 1
Worker: 2
Worker: 3
Worker: 4


In [5]:
%%time
threads = []
for i in range(5):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

Worker: 0
Worker: 1
Worker: 2
Worker: 3
Worker: 4
CPU times: user 1.3 ms, sys: 1.17 ms, total: 2.47 ms
Wall time: 1.84 ms


Despite the GIL, threads wont get in each other's way if they are idle.

In [7]:
#%%time
import logging
import random
import time

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)

import threading

def worker(num):
    """thread worker function"""
    
    sleep_time = random.randint(1,5)
    logging.debug('worker: {0} sleeping for {1} s, name: {2}'
                   .format(num,sleep_time,threading.current_thread().getName()))
    time.sleep(sleep_time)
    logging.debug('done')
    return

threads = []
for i in range(2):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

(Thread-21) worker: 0 sleeping for 5 s, name: Thread-21
(Thread-22) worker: 1 sleeping for 1 s, name: Thread-22
(Thread-22) done
(Thread-21) done


`.join()`

In [10]:
%%time
# not very parallel ... 
threads = []
for i in range(2):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()
    t.join() # this waits for the thread to finish

(Thread-25) worker: 0 sleeping for 3 s, name: Thread-25
(Thread-25) done
(Thread-26) worker: 1 sleeping for 3 s, name: Thread-26
(Thread-26) done


CPU times: user 4.23 ms, sys: 2.12 ms, total: 6.35 ms
Wall time: 6.01 s


In [14]:
%%time
threads = []
for i in range(2):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)

print("waiting around a bit, then starting threads",flush=True)
#time.sleep(2)

# dont have to start a thread immediately after creating them
for t in threads:
    t.start() 

for t in threads:
    t.join() # this waits for the thread to finish
    
print("Done with all the threads!")

waiting around a bit, then starting threads


(Thread-33) worker: 0 sleeping for 2 s, name: Thread-33
(Thread-34) worker: 1 sleeping for 4 s, name: Thread-34
(Thread-33) done
(Thread-34) done


Done with all the threads!
CPU times: user 5.05 ms, sys: 2.21 ms, total: 7.26 ms
Wall time: 4.01 s


A few things:

- `logging` is "thread-safe" -- so different threads can write to the log file without causing issues
- you can always get a handle to the current thread with `threading.current_thread()`

You can delay the start of the execution of a thread with `Timer`

```python
threading.Timer(interval, function, args=None, kwargs=None)
```

In [15]:
threads = []
for i in range(2):
    r = random.randint(1,5)
    t = threading.Timer(r, worker, args=(i,))
    threads.append(t)
    logging.debug("starting {0} with delay {1}"
                  .format(t.getName(),r))
    threads[-1].start()

(MainThread) starting Thread-35 with delay 5
(MainThread) starting Thread-36 with delay 2
(Thread-36) worker: 1 sleeping for 4 s, name: Thread-36
(Thread-35) worker: 0 sleeping for 5 s, name: Thread-35
(Thread-36) done
(Thread-35) done


You can share variables (safely) between threads with a `queue`:

In [17]:
from queue import Queue

q = Queue()

def worker2(num):
    sleep_time = random.randint(1,5)
    
    logging.debug('worker: {0} sleeping for {1} s, name: {2}'
                   .format(num,sleep_time,threading.current_thread().getName()))
    time.sleep(sleep_time)

    if q.empty():
        q.put(sleep_time)
        logging.debug("initiated q = {0}".format(sleep_time))
    else:
        var = q.get()
        logging.debug("var {0}".format(var))
        q.put(sleep_time + var)
        logging.debug("added {0} to the q".format(sleep_time))
        
    logging.debug('done')
    return

threads = []
for i in range(2):
    t = threading.Thread(target=worker2, args=(i,))
    threads.append(t)
    t.start()

(Thread-39) worker: 0 sleeping for 4 s, name: Thread-39
(Thread-40) worker: 1 sleeping for 5 s, name: Thread-40
(Thread-39) initiated q = 4
(Thread-39) done
(Thread-40) var 4
(Thread-40) added 5 to the q
(Thread-40) done


In [18]:
q.get()

9

Threads can also signal each other with `Event` and can thresholds for the numbers of finished threads can be created with `Barrier`. There are low-level primiatives (pushed to the UNIX \_pthreads level) called `locks` and `semaphores` that we'll not bother with here.

Threading can be done with objects. You can subclass `threading.Thread` and create your own threads that know how to run.

In [19]:
os.popen("ping -q -c2 cnn.com","r").readlines()

['PING cnn.com (157.166.226.25): 56 data bytes\n',
 '\n',
 '--- cnn.com ping statistics ---\n',
 '2 packets transmitted, 2 packets received, 0.0% packet loss\n',
 'round-trip min/avg/max/stddev = 64.440/64.594/64.748/0.154 ms\n']

In [22]:
# adapted from http://www.python-course.eu/threads.php
import os, re, threading

received_packages = re.compile(r"(\d) packets received")

class ip_check(threading.Thread):
  
    def __init__ (self,ip):
      threading.Thread.__init__(self)
      self.ip = ip
      self.__successful_pings = -1
   
    def run(self):
      ping_out = os.popen("ping -q -c2 "+self.ip,"r")
      while True:
        lines = ping_out.readlines()
        if not lines or len(lines) < 3: 
            break
        n_received = re.findall(received_packages,lines[3])
        if n_received:
           self.__successful_pings = int(n_received[0])
        
    def status(self):
      if self.__successful_pings == 0:
         return "has no response"
      elif self.__successful_pings == 1:
         return "is alive, but 50 % package loss"
      elif self.__successful_pings == 2:
         return "is alive"
      else:
         return "not reachable"

check_results = []
for ip in ["google.com","slashdot.com","berkeley.edu","blasaskaslkas.org", "cnn.com", "danlurie.org"]:
   current = ip_check(ip)
   check_results.append(current)
   current.start()

for el in check_results:
   el.join()
   print("Status of", el.ip,el.status())

Status of google.com is alive
Status of slashdot.com is alive
Status of berkeley.edu is alive
Status of blasaskaslkas.org not reachable
Status of cnn.com is alive
Status of danlurie.org is alive


### Breakout

Using threading, grab the titles of random 10 wikipedia webpages using https://en.wikipedia.org/wiki/Special:Random. Count the total number of characters returned over all 10 pages.

In [32]:
import requests

In [ ]:
q = Queue()

def title_bot(url):
    logging.debug('worker: {0} sleeping for {1} s, name: {2}'
                   .format(num,sleep_time,threading.current_thread().getName()))
    resp = requests.get("https://en.wikipedia.org/wiki/Special:Random")
    site_title = BeautifulSoup(resp.text, 'html.parser').title.string.split("- Wikipedia")[0]
    resp_len = len(resp.text)
    
    if q.empty():
        q.put(sleep_time)
        logging.debug("initiated q = {0}".format(sleep_time))
    else:
        var = q.get()
        logging.debug("var {0}".format(var))
        q.put(sleep_time + var)
        logging.debug("added {0} to the q".format(sleep_time))
        
    logging.debug('done')
    return

threads = []
for i in range(2):
    t = threading.Thread(target=worker2, args=(i,))
    threads.append(t)
    t.start()

In [38]:
resp = requests.get("https://en.wikipedia.org/wiki/Special:Random")

(MainThread) Starting new HTTPS connection (1): en.wikipedia.org
(MainThread) "GET /wiki/Special:Random HTTP/1.1" 302 20
(MainThread) "GET /wiki/Indonesia%E2%80%93Palestine_relations HTTP/1.1" 200 15846


In [39]:
BeautifulSoup(resp.text, 'html.parser').title.string.split("- Wikipedia")[0]

'Indonesia–Palestine relations '

In [35]:
resp.text

'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content="Search the world\'s information, including webpages, images, videos and more. Google has many special features to help you find exactly what you\'re looking for." name="description"><meta content="noodp" name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/logos/doodles/2016/rodolfo-guzman-huerta-el-santos-99th-birthday-5630362233339904-hp.jpg" itemprop="image"><meta content="Rodolfo Guzmán Huerta (El Santo)&#8217;s 99th Birthday #GoogleDoodle" property="og:description"><meta content="http://www.google.com/logos/doodles/2016/rodolfo-guzman-huerta-el-santos-99th-birthday-5630362233339904.2-thp.jpg" property="og:image"><meta content="487" property="og:image:width"><meta content="210" property="og:image:height"><title>Google</title><script>(function(){window.google={kEI:\'f5zlV5eDMM6KjwOWyKr4Aw\',kEXPI:\'750721,1351633,1351728,3700261,4028875,4

In [ ]:
BeautifulSoup.fe

Aside: for asynchronous I/O tasks you might consider using an event loop. Use the built in `asyncio` and, for gathering webpages, use `aiohttp` (http://aiohttp.readthedocs.io/en/stable/).

In [31]:
import asyncio
from aiohttp import ClientSession
from bs4 import BeautifulSoup

async def fetch(url, session):
    async with session.get(url) as response:
        return await response.read()

async def run(loop,  r):
    url = "https://en.wikipedia.org/wiki/Special:Random"
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    async with ClientSession() as session:
        for i in range(r):
            task = asyncio.ensure_future(fetch(url, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
    
    for resp in responses:
        site_title = BeautifulSoup(resp, 'html.parser').title.string.split("- Wikipedia")[0]
        resp_len = len(resp)
        print(site_title, resp_len)
    
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run(loop, 10))
loop.run_until_complete(future)

Elaphrus lapponicus  28833
Velleman (surname)  23312
Kaboom! (video game)  59547
Gorni Lom  37573
Belmont (Sutton) railway station  87696
Naruto Shippūden 3D: The New Era  53442
The Fountain Inn, Gloucester  33225
Pragmalji II  32831
Escambia map turtle  81702
Live at Wembley '86  80006
